In [1]:
from __future__ import division
import pandas as pd
import numpy as np
from TumorTruths import TumorTruths
from scipy.stats import mode
from sklearn.metrics import f1_score
# from sklearn import cluster
# from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import manhattan_distances
# from sklearn.metrics.pairwise import euclidean_distances
from DistanceClustering import DistanceClustering

In [2]:
svnCalls = pd.read_csv('SNVCalls_IS4.txt', delimiter='\t', dtype={'CHROM':pd.np.str})
truths = TumorTruths('synthetic.challenge.set4.tumour.25pctmasked.truth.vcf')

In [3]:
resultsArray = truths.GetTruths(svnCalls)
svnCalls = svnCalls[svnCalls.columns[3:-1]]

In [4]:
class SomaticMutationClassifier:
    
    def __init__(self, num_clusters, inflection_point):
        self.num_clusters = num_clusters
        self.transformToPredictions = np.vectorize(lambda x: 1 if x >= inflection_point else 0) 
    
    def fit(self, df):
        kMediansClustering = DistanceClustering(num_clusters, manhattan_distances, np.median)
        self.clusters, self.centers = kMediansClustering.fit(df.T.values)
        
    def predict(self, df):
        predictions = np.zeros(len(df))
        callersInPipeline = list()
        
        for i in range(0, self.num_clusters):
            weight = len([1 for num in self.clusters  if num == i])
            distances = manhattan_distances(df.T, self.centers[i])
            bestIndex = np.argmin(distances)
            bestCallerName = df.columns[bestIndex]
            representativeCaller = df[bestCallerName]
            callersInPipeline.append(bestCallerName)
            predictions += weight * representativeCaller

        predictions /= len(self.clusters)
        predictions = self.transformToPredictions(predictions) 
        
        return predictions, callersInPipeline

num_clusters = 5
somaticMutationClassifier = SomaticMutationClassifier(num_clusters, 0.50)
somaticClustering.fit(svnCalls)
predictions, callersInPipeline = somaticMutationClassifier.predict(svnCalls)

In [5]:
f1_score(predictions, resultsArray)

0.87254795315276124

In [6]:
mostCommonPredictions = mode(svnCalls, axis=1)[0]
f1_score(mostCommonPredictions, resultsArray)

0.83674707935062975